In [ ]:
# !pip3 uninstall -y tensorflow-federated-nightly


In [ ]:
# !pip3 install tensorflow_federated

In [1]:
import os
from operator import itemgetter
import collections

import matplotlib.pylab as plt
# %matplotlib widget
%matplotlib inline

import tensorflow as tf
# tf.random.set_seed(99)
import tensorflow_federated as tff
import numpy as np
import nest_asyncio
nest_asyncio.apply()
#tf.compat.v1.enable_v2_behavior() # https://www.tensorflow.org/api_docs/python/tf/compat/v1/enable_v2_behavior

print(f'Tensorflow version: {tf.__version__}')
print(f'Tensorflow Federated version: {tff.__version__}')

Tensorflow version: 2.7.0
Tensorflow Federated version: 0.19.0


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.per_process_gpu_memory_fraction = 0.85 #메모리 사용율 제한
        #logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [3]:
# Global variables
# Setup scripts (or notebook)
# Global variables
# Setup scripts (or notebook)
IMG_DATA = './equal_train'
IMG_SHAPE = (224, 224)
BATCH_SIZE = 32
CLASSES = ['acne', 'melanoma', 'psoriasis', 'wart']


In [4]:
%%time
# prepare dataset
dataset_root = os.path.abspath(os.path.expanduser(IMG_DATA))
print(f'Dataset root: {dataset_root}')

img_gen_op = {'classes': CLASSES, 'target_size': IMG_SHAPE, 'batch_size': BATCH_SIZE}
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

def gen_fn(args):
    data_path = args.decode('utf-8')
    return image_generator.flow_from_directory(data_path,
                                               **img_gen_op)

dataset_size = dict()
queue = [dataset_root]
while queue:
    path = queue.pop(0)
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_dir():
                queue.append(entry.path)
            if entry.is_file():
                name = os.path.basename(os.path.dirname(os.path.dirname(entry.path)))
                dataset_size[name] = dataset_size.get(name, 0) + 1

dataset_dict = dict()
with os.scandir(dataset_root) as it:
    for entry in it:
        if entry.is_dir():
            name = os.path.basename(entry.path)
            ds = tf.data.Dataset.from_generator(gen_fn,
                                                args=[entry.path],
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=(tf.TensorShape([None, 224, 224, 3]), 
                                                               tf.TensorShape([None, len(CLASSES)]))
                                               )
            dataset_dict[name] = ds

Dataset root: /home/dnlab/Desktop/20211024T133959/equal_train
CPU times: user 375 ms, sys: 211 ms, total: 586 ms
Wall time: 587 ms


In [5]:
ds_series_batch = dataset_dict['0'].shuffle(20).padded_batch(6)

ids, sequence_batch = next(iter(ds_series_batch))

print(sequence_batch.numpy())

Found 180 images belonging to 4 classes.
[[[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1

In [6]:
def client_fn(client_id):
    return dataset_dict[client_id]

client_data = tff.simulation.datasets.ClientData.from_clients_and_fn(
                client_ids=list(dataset_dict.keys()),
                 create_tf_dataset_for_client_fn=client_fn)

train_ids = list(dataset_dict.keys())
train_ids.remove('0')
# train_ids = ['1'] ## for experiment client each
dataset = [(client_data.create_tf_dataset_for_client(x), dataset_size[x]) for x in train_ids]

In [7]:
example_dataset = (client_data.create_tf_dataset_for_client(client_data.client_ids[0]),
                   dataset_size[client_data.client_ids[0]])
print(example_dataset)

(<FlatMapDataset shapes: ((None, 224, 224, 3), (None, 4)), types: (tf.float32, tf.float32)>, 120)


In [8]:
# Algorithm
import statistics
# take_value = statistics.median(dataset_size.values())
take_value = None

In [9]:
def preprocess(dataset, take_value=None):
#     return dataset[0].take(np.ceil(dataset[1]/BATCH_SIZE))
    if take_value is None:
        take_value = dataset[1]
    else:
        take_value = min(take_value, dataset[1])
    return dataset[0].take(np.ceil(take_value/BATCH_SIZE))
    
preprocessed_example_dataset = preprocess(example_dataset, take_value)
sample_batch = tf.nest.map_structure(lambda x: x.numpy(), iter(preprocessed_example_dataset).next())
print(sample_batch[0].shape, sample_batch[1].shape)

Found 120 images belonging to 4 classes.
(32, 224, 224, 3) (32, 4)


In [10]:
federated_dataset = [preprocess(x, take_value) for x in dataset]

In [11]:
# For evaluation
test_dataset = (client_data.create_tf_dataset_for_client('0'), dataset_size['0'])
federated_test_data = [preprocess(test_dataset, take_value)]

In [12]:
print(type(client_data.create_tf_dataset_for_client('0')))

<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>


In [13]:
model_name = "mobilenet_v3_large_075_224" # @param ['efficientnetv2-s', 'efficientnetv2-m', 'efficientnetv2-l', 'efficientnetv2-s-21k', 'efficientnetv2-m-21k', 'efficientnetv2-l-21k', 'efficientnetv2-xl-21k', 'efficientnetv2-b0-21k', 'efficientnetv2-b1-21k', 'efficientnetv2-b2-21k', 'efficientnetv2-b3-21k', 'efficientnetv2-s-21k-ft1k', 'efficientnetv2-m-21k-ft1k', 'efficientnetv2-l-21k-ft1k', 'efficientnetv2-xl-21k-ft1k', 'efficientnetv2-b0-21k-ft1k', 'efficientnetv2-b1-21k-ft1k', 'efficientnetv2-b2-21k-ft1k', 'efficientnetv2-b3-21k-ft1k', 'efficientnetv2-b0', 'efficientnetv2-b1', 'efficientnetv2-b2', 'efficientnetv2-b3', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'bit_s-r50x1', 'inception_v3', 'inception_resnet_v2', 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152', 'nasnet_large', 'nasnet_mobile', 'pnasnet_large', 'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

model_handle_map = {
  "efficientnetv2-s": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2",
  "efficientnetv2-m": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_m/feature_vector/2",
  "efficientnetv2-l": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/feature_vector/2",
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/feature_vector/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/feature_vector/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/feature_vector/2",
  "efficientnetv2-s-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/feature_vector/2",
  "efficientnetv2-m-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/feature_vector/2",
  "efficientnetv2-l-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_l/feature_vector/2",
  "efficientnetv2-xl-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2",
  "efficientnetv2-b0-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/feature_vector/2",
  "efficientnetv2-b1-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b1/feature_vector/2",
  "efficientnetv2-b2-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b2/feature_vector/2",
  "efficientnetv2-b3-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2",
  "efficientnetv2-b0": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",
  "efficientnetv2-b1": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/feature_vector/2",
  "efficientnetv2-b2": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/feature_vector/2",
  "efficientnetv2-b3": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/feature_vector/2",
  "efficientnet_b0": "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
  "efficientnet_b1": "https://tfhub.dev/tensorflow/efficientnet/b1/feature-vector/1",
  "efficientnet_b2": "https://tfhub.dev/tensorflow/efficientnet/b2/feature-vector/1",
  "efficientnet_b3": "https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1",
  "efficientnet_b4": "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1",
  "efficientnet_b5": "https://tfhub.dev/tensorflow/efficientnet/b5/feature-vector/1",
  "efficientnet_b6": "https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1",
  "efficientnet_b7": "https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1",
  "bit_s-r50x1": "https://tfhub.dev/google/bit/s-r50x1/1",
  "inception_v3": "https://tfhub.dev/google/imagenet/inception_v3/feature-vector/4",
  "inception_resnet_v2": "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature-vector/4",
  "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/feature-vector/4",
  "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/feature-vector/4",
  "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/feature-vector/4",
  "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4",
  "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/feature-vector/4",
  "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/feature-vector/4",
  "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/4",
  "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/4",
  "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/feature_vector/4",
  "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
  "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
  "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
  "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
  "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
  "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
  "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
}

model_image_size_map = {
  "efficientnetv2-s": 384,
  "efficientnetv2-m": 480,
  "efficientnetv2-l": 480,
  "efficientnetv2-b0": 224,
  "efficientnetv2-b1": 240,
  "efficientnetv2-b2": 260,
  "efficientnetv2-b3": 300,
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "efficientnetv2-s-21k-ft1k": 384,
  "efficientnetv2-m-21k-ft1k": 480,
  "efficientnetv2-l-21k-ft1k": 480,
  "efficientnetv2-xl-21k-ft1k": 512,
  "efficientnetv2-b0-21k-ft1k": 224,
  "efficientnetv2-b1-21k-ft1k": 240,
  "efficientnetv2-b2-21k-ft1k": 260,
  "efficientnetv2-b3-21k-ft1k": 300, 
  "efficientnet_b0": 224,
  "efficientnet_b1": 240,
  "efficientnet_b2": 260,
  "efficientnet_b3": 300,
  "efficientnet_b4": 380,
  "efficientnet_b5": 456,
  "efficientnet_b6": 528,
  "efficientnet_b7": 600,
  "inception_v3": 299,
  "inception_resnet_v2": 299,
  "nasnet_large": 331,
  "pnasnet_large": 331,
}

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name, 224)

print(f"Selected model: {model_name} : {model_handle}")

IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 32#@param {type:"integer"}

Selected model: mobilenet_v3_large_075_224 : https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5
Input size (224, 224)


In [14]:
# def build_dataset(subset):
#   return tf.keras.preprocessing.image_dataset_from_directory(
#       '../test',
#       validation_split=.20,
#       subset=subset,
#       label_mode="categorical",
#       # Seed needs to provided when using validation_split and shuffle = True.
#       # A fixed seed is used so that the validation set is stable across runs.
#       seed=123,
#       image_size=IMAGE_SIZE,
#       batch_size=1)

# train_ds = build_dataset("training")
class_names = ['acne','melanoma','psoriasis','wart']
# class_names = tuple(train_ds.class_names)
# train_size = train_ds.cardinality().numpy()
# train_ds = train_ds.unbatch().batch(BATCH_SIZE)
# train_ds = train_ds.repeat()

# normalization_layer = tf.keras.layers.Rescaling(1. / 255)
# preprocessing_model = tf.keras.Sequential([normalization_layer])
# do_data_augmentation = False #@param {type:"boolean"}
# if do_data_augmentation:
#   preprocessing_model.add(
#       tf.keras.layers.RandomRotation(40))
#   preprocessing_model.add(
#       tf.keras.layers.RandomTranslation(0, 0.2))
#   preprocessing_model.add(
#       tf.keras.layers.RandomTranslation(0.2, 0))
#   # Like the old tf.keras.preprocessing.image.ImageDataGenerator(),
#   # image sizes are fixed when reading, and then a random zoom is applied.
#   # If all training inputs are larger than image_size, one could also use
#   # RandomCrop with a batch size of 1 and rebatch later.
#   preprocessing_model.add(
#       tf.keras.layers.RandomZoom(0.2, 0.2))
#   preprocessing_model.add(
#       tf.keras.layers.RandomFlip(mode="horizontal"))
# train_ds = train_ds.map(lambda images, labels:
#                         (preprocessing_model(images), labels))

# val_ds = build_dataset("validation")
# valid_size = val_ds.cardinality().numpy()
# val_ds = val_ds.unbatch().batch(BATCH_SIZE)
# val_ds = val_ds.map(lambda images, labels:
#                     (normalization_layer(images), labels))

In [15]:
do_fine_tuning = False #@param {type:"boolean"}

In [16]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.7.0
Hub version: 0.9.0
GPU is available


In [17]:
def create_keras_model():
    model = tf.keras.Sequential([
        # Explicitly define the input shape so the model can be properly
        # loaded by the TFLiteConverter
        tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
        hub.KerasLayer(model_handle, trainable=do_fine_tuning),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(len(class_names),
                              kernel_regularizer=tf.keras.regularizers.l2(0.0001))
    ])
    
    return model

In [18]:
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(keras_model, 
                                         input_spec=preprocessed_example_dataset.element_spec,
#                                        dummy_batch=sample_batch,
                                         loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                                         metrics=[tf.keras.metrics.CategoricalAccuracy()])


In [19]:
iterative_process = tff.learning.build_federated_averaging_process(model_fn,
                           client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
                           server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [20]:
str(iterative_process.initialize.type_signature)

'( -> <model=<trainable=<float32[1280,4],float32[4]>,non_trainable=<float32[16],float32[72],float32[72],float32[96],float32[32],float32[1,1,144,64],float32[528],float32[32],float32[384],float32[528],float32[72],float32[192],float32[192],float32[5,5,528,1],float32[528],float32[72],float32[64],float32[384],float32[528],float32[1,1,184,720],float32[1,1,64,24],float32[72],float32[5,5,96,1],float32[1,1,32,192],float32[64],float32[144],float32[1,1,384,88],float32[1,1,96,24],float32[1,1,144,64],float32[144],float32[3,3,528,1],float32[32],float32[1,1,96,32],float32[1,1,160,64],float32[384],float32[120],float32[3,3,72,1],float32[3,3,3,16],float32[96],float32[160],float32[384],float32[1,1,88,528],float32[720],float32[3,3,64,1],float32[96],float32[64],float32[720],float32[1,1,24,72],float32[5,5,96,1],float32[64],float32[64],float32[1,1,88,528],float32[720],float32[1,1,16,16],float32[64],float32[144],float32[1,1,64,384],float32[96],float32[1,1,528,136],float32[5,5,720,1],float32[64],float32[88],fl

In [21]:
evaluation = tff.learning.build_federated_evaluation(model_fn)


In [22]:
print(evaluation)

In [23]:
state = iterative_process.initialize()

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [24]:
NUM_ROUNDS = 1000
MAX_STD = 0.001
loss = list()
accuracy = list()

val_loss = list()
val_accuracy = list()

max = 0
import shutil
import os

for round_num in range(1, NUM_ROUNDS+1):
    state, metrics = iterative_process.next(state, federated_dataset)
    val_metrics = evaluation(state.model, federated_test_data)
    loss.append(metrics['train']['loss'])
    accuracy.append(metrics['train']['categorical_accuracy'])
    val_loss.append(val_metrics['loss'])
    val_accuracy.append(val_metrics['categorical_accuracy'])
    print(f'round: {round_num:2d}, loss: {metrics["train"]["loss"]}, test_accuracy: {val_metrics["categorical_accuracy"]}')
    #가장 높은 확률일 때 모델 세이브
    if(val_metrics["categorical_accuracy"] > max):
        create_keras_model().save_weights("ckpt")
        
#     create_keras_model().save(create_keras_model().set_weights(create_keras_model().get_weights()))
    #         create_keras_model().save('./save_model') #keras pb
    
    if len(val_loss) > 3 and np.std(val_loss[-3:]) < MAX_STD:
        break

Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
round:  1, loss: 1.4418507814407349, test_accuracy: 0.4722222089767456
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
round:  2, loss: 1.0716438293457031, test_accuracy: 0.5944444537162781
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
round:  3, loss: 0.8624536991119385, test_accuracy: 0.6666666865348816
Found 120 images belonging to 4 classes.
Found 12

Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.Found 120 images belonging to 4 classes.

Found 180 images belonging to 4 classes.
round: 27, loss: 0.23055541515350342, test_accuracy: 0.8666666746139526
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
round: 28, loss: 0.22346438467502594, test_accuracy: 0.8722222447395325
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
round: 29, loss: 0.21684692800045013, test_accuracy: 0.8777777552604675
Found 120 images belonging to 4 classes.
Found

Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
round: 53, loss: 0.13913168013095856, test_accuracy: 0.9222221970558167
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
round: 54, loss: 0.14617817103862762, test_accuracy: 0.9222221970558167
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
round: 55, loss: 0.1409335732460022, test_accuracy: 0.9222221970558167
Found 120 images belonging to 4 classes.
Found 

In [ ]:
create_keras_model().set_weights(create_keras_model().get_weights())

In [ ]:
##################################   pred ##################################################

#마지막 레이어 확률 값 
# probability_model = tf.keras.Sequential([create_keras_model(), 
#                                          tf.keras.layers.Softmax()])
from keras.models import load_model
# model = create_keras_model()
# model.set_weights(state.model.trainable)
# model.predict()
probability_model = tf.keras.models.load_model('./save')
print(probability_model)


In [ ]:
print(probability_model.predict(federated_test_data[0]))
predictions = probability_model.predict(federated_test_data[0])

In [ ]:
#테스트 데이터 라벨 맞춘 개수
a = 0
b = 0
c = 0
d = 0
for i in range(180):
    print(predictions[i])
    n = np.argmax(predictions[i])
    # print(np.argmax(predictions[i]))
    if n==0:
      a = a+1
    elif n==1 :
      b = b+1
    elif n==2 :
      c = c+1
    elif n==3:
      d = d+1
print(a," ",b," ",c," ",d)

In [ ]:
# #실제 라벨 리스트 만들기
# actual_value = tf.keras.Sequential([create_keras_model(),tf.keras.layers.InputLayer()])
# actuals = actual_value.predict(federated_test_data[0])
# print(actuals)
# a = 0
# b = 0
# c = 0
# d = 0
# for i in range(180):
#     print(actuals[i])
#     n = np.argmax(actuals[i])
#     # print(np.argmax(predictions[i]))
#     if n==0:
#       a = a+1
#     elif n==1 :
#       b = b+1
#     elif n==2 :
#       c = c+1
#     elif n==3:
#       d = d+1
# print(a," ",b," ",c," ",d)

In [ ]:
##################################   pred ##################################################

#마지막 레이어 확률 값 
# probability_model = tf.keras.Sequential([create_keras_model(), 
#                                          tf.keras.layers.Softmax()])
from keras.models import load_model
# model = create_keras_model()
# model.set_weights(state.model.trainable)
# model.predict()
probability_model = tf.keras.models.load_model('./save_model')
print(probability_model)


In [ ]:
print(probability_model.predict(federated_test_data[0]))
predictions = probability_model.predict(federated_test_data[0])

In [ ]:
#테스트 데이터 라벨 맞춘 개수
a = 0
b = 0
c = 0
d = 0
for i in range(180):
    print(predictions[i])
    n = np.argmax(predictions[i])
    # print(np.argmax(predictions[i]))
    if n==0:
      a = a+1
    elif n==1 :
      b = b+1
    elif n==2 :
      c = c+1
    elif n==3:
      d = d+1
print(a," ",b," ",c," ",d)

In [ ]:
####################### 데이터셋 라벨 확인할 것 ##############################
# print(test_dataset.index('acne'))
ac = probability_model.predict(federated_dataset[1])
a = 0
b = 0
c = 0
d = 0
for i in range(143):
    print(ac[i])
    n = np.argmax(ac[i])
    print(np.argmax(ac[i]))
    if n==0:
      a = a+1
    elif n==1 :
      b = b+1
    elif n==2 :
      c = c+1
    elif n==3:
      d = d+1
print(a," ",b," ",c," ",d)


In [ ]:
##############################################################################################

In [ ]:
fig1 = plt.figure(figsize=(8, 8))
ax1 = fig1.add_subplot(2, 1, 1)
ax1.plot(accuracy, label='Training Accuracy')
ax1.plot(val_accuracy, label='Validation Accuracy')
ax1.legend(loc='lower right')
ax1.set_ylabel('Accuracy')
ax1.set_ylim([0, 1])
ax1.set_title('Training and Validation Accuracy')

# ax2 = fig1.add_subplot(2, 1, 2)
# ax2.plot(loss, label='Training Loss')
# ax2.plot(val_loss, label='Validation Loss')
# ax2.legend(loc='upper right')
# ax2.set_ylabel('Cross Entropy')
# ax2.set_ylim([0,max(ax2.get_ylim())])
# ax2.set_title('Training and Validation Loss')
# ax2.set_xlabel('epoch')

In [ ]:
import pickle

with open('output.pickle', 'wb') as f:
    pickle.dump((accuracy, val_accuracy, loss, val_loss), f)
with open('output.pickle', 'rb') as f:
    print(max(pickle.load(f)[1]))
print(max(val_accuracy))



# import pandas as pd

# print("pandas version: ", pd.__version__)
# pd.set_option('display.max_row', 5000)
# pd.set_option('display.max_columns', 1000)

In [ ]:
# df = pd.DataFrame(iterative_process.initialize())

print(iterative_process.initialize())

In [ ]:
print(iterative_process.next(state, federated_dataset))

In [ ]:
print(state.model)

In [ ]:
print(val_metrics['categorical_accuracy'])

In [ ]:
# numpy 배열이 일부만 표시됨.

 

# 해결방법

# import sys

# import numpy as np

# np.set_printoptions(threshold=sys.maxsize)

 

# 옵션을 바꿔주면 배열 전체가 표시됨.


In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
# import seaborn as sns
 
# cm2 = confusion_matrix(comp_df['실제값'],comp_df['예측값'])
# cmdf2 = DataFrame(cm2, index=['실제값(N)', '실제값(P)'], columns=['예측값(N)', '예측값(P)'])
# cmdf2


In [ ]:
print(state)

In [ ]:
print(federated_dataset)

In [ ]:
#모델 저장
model = create_keras_model()
model.save('./my_model.h5')

In [ ]:
#pb -> tflite 
from tensorflow import keras
#허브모델 사용시
model = tf.keras.models.load_model('./save_model/', custom_objects={'KerasLayer':hub.KerasLayer})
#기본 모델 사용시
# model = tf.keras.models.load_model('./my_model.h5')
export_path = './pb'
model.save(export_path, save_format="tf")

In [ ]:
################################3 누군가 사용한 UINT8만드는 converter
def representative_dataset_gen():
    for _ in range(10):
        dummy_image = tf.random.uniform([1, 224, 224, 3], 0., 255., dtype=tf.float32)
        dummy_image = dummy_image / 127.5 - 1
        yield [dummy_image]
saved_model_dir = './save/'
converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()
open('./mobileNetv2_uint8.tflite', 'wb').write(tflite_model)

# print('TensorFlow Model is  {} bytes'.format(tf_model_size))
# print('TFLite Model is      {} bytes'.format(tflite_model_size))
# print('Post training int8 quantization saves {} bytes'.format(tf_model_size-tflite_model_size))

In [ ]:
saved_model_dir = './save/'
converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

tflite_model = converter.convert()
open('./mobileNetv2.tflite', 'wb').write(tflite_model)

In [63]:
import os
import numpy as np
import tensorflow as tf
import importlib

IMG_SIZE = 224
NUM_FEATURES = 7 * 7 * 1280
NUM_CLASSES = 4

class TransferLearningModel(tf.Module):
    def __init__(self, learning_rate=0.001):
        self.num_features = NUM_FEATURES
        self.num_classes = NUM_CLASSES

        # trainable weights and bias for softmax
        self.ws = tf.Variable(
            tf.zeros((self.num_features, self.num_classes)),
            name='ws',
            trainable=True)
        self.bs = tf.Variable(
            tf.zeros((1, self.num_classes)), name='bs', trainable=True)

        # base model
        self.base = tf.keras.applications.MobileNetV2(
            input_shape=(IMG_SIZE, IMG_SIZE, 3),
            alpha=1.0,
            include_top=False,
            weights= "imagenet")
        # loss function and optimizer
        self.loss_fn = tf.keras.losses.CategoricalCrossentropy()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    @tf.function(input_signature=[
          tf.TensorSpec([None, IMG_SIZE, IMG_SIZE, 3], tf.float32),
      ])
    def load(self, feature):
        x = tf.keras.applications.mobilenet_v2.preprocess_input(
            tf.multiply(feature, 255))
        bottleneck = tf.reshape(
            self.base(x, training=False), (-1, self.num_features))
        return {'bottleneck': bottleneck}

    @tf.function(input_signature=[
          tf.TensorSpec([None, NUM_FEATURES], tf.float32),
          tf.TensorSpec([None, NUM_CLASSES], tf.float32),
      ])
    def train(self, bottleneck, label):
        with tf.GradientTape() as tape:
            logits = tf.matmul(bottleneck, self.ws) + self.bs
            prediction = tf.nn.softmax(logits)
            loss = self.loss_fn(prediction, label)
        gradients = tape.gradient(loss, [self.ws, self.bs])
        self.optimizer.apply_gradients(zip(gradients, [self.ws, self.bs]))
        result = {'loss': loss}
        for grad in gradients:
            result[grad.name] = grad
        return result

    @tf.function(input_signature=[
          tf.TensorSpec([None, IMG_SIZE, IMG_SIZE, 3], tf.float32)
          ])
    def infer(self, feature):
        x = tf.keras.applications.mobilenet_v2.preprocess_input(
            tf.multiply(feature, 255))
        bottleneck = tf.reshape(
            self.base(x, training=False), (-1, self.num_features))
        logits = tf.matmul(bottleneck, self.ws) + self.bs
        return {'output': tf.nn.softmax(logits)}

def convert_and_save(saved_model_dir='saved_model'):
    model = TransferLearningModel()
#학습이 안되어있음.얹을 방법이 없나???
    tf.saved_model.save(
          model,
          saved_model_dir,
          signatures={
              'load': model.load.get_concrete_function(),
              'train': model.train.get_concrete_function(),
              'infer': model.infer.get_concrete_function(),   
          })

    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    converter.target_spec.supported_ops = [
          tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
          tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
      ]
    converter.experimental_enable_resource_variables = True
    tflite_model = converter.convert()

    model_file_path = os.path.join('model.tflite')
    with open(model_file_path, 'wb') as model_file:
        model_file.write(tflite_model)

if __name__ == '__main__':
    convert_and_save()

2021-12-07 17:00:45.797985: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets
2021-12-07 17:01:00.826365: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2021-12-07 17:01:00.826391: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2021-12-07 17:01:00.826396: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2021-12-07 17:01:03.845508: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1891] TFLite interpreter needs to link Flex delegate in order to run the model since it contains the following Select TFop(s):
Flex ops: FlexBroadcastGradientArgs
Details:
	tf.BroadcastGradientArgs(tensor<2xi32>, tensor<2xi32>) -> (tensor<?xi32>, tensor<?xi32>) : {device = ""}
See instructions: https://www.tensorflow.org/lite/guide/ops_select
